Point Mutaul Information (PMI) 은 두 변수의 상관성을 측정하는 방법 중 하나입니다.

Brief review of Point Mutial Information

확률에서 두 확률 분포가 서로 독립인가 (두 확률은 서로 상관이 없는가)의 조건은 joint probability 와 각각의 marginal probability 의 곱의 비율이 1인가 입니다. ..

## Implementing PMI with dense matrix (numpy.ndarray)

Numpy 의 dense matrix 로 표현되는 작은 행렬을 이용하여 matrix handling 에 대하여 연습합니다.
눈으로 값의 변화를 확인하며 행렬을 다루는 방법을 익힌 뒤, sparse matrix에 응용합니다.

In [1]:
import numpy as np

x = np.array([[3, 0, 0], [0, 2, 0], [1, 0, 1], [2, 0, 1]])

p(x) 와 p(y)를 계산합니다.
x.sum(axis=0) 은 row sum 이며, x.sum(axis=1) 은 column sum 입니다.
모든 columns 을 하나로 합치면 각 row 의 sum 이 계산됩니다. 
이 값을 행렬 전체의 합인 x.sum() 으로 나누면 p(x) 를 얻을 수 있습니다. 

In [2]:
# marginal probability
px = x.sum(axis=1) / x.sum()
py = x.sum(axis=0) / x.sum()

print(px)
print(py)

[0.3 0.2 0.2 0.3]
[0.6 0.2 0.2]


p(x, y)를 계산하기 위해서는 x를 x.sum()으로 나눕니다

In [3]:
# convert x to probability matrix
pxy = x / x.sum()
print(pxy)

[[0.3 0.  0. ]
 [0.  0.2 0. ]
 [0.1 0.  0.1]
 [0.2 0.  0.1]]


p(x, y) 를 p(x) 로 나누기 위해서는 행렬곱을 이용할 수 있습니다.
p(x) 와 p(y)를 diagonal matrix 로 만듭니다.
p(x) 의 i번째 값을 diagonal matrix 의 (i, i) 의 값입니다. 
이를 위해 numpy.diag 를 이용합니다.
numpy.diag 는 array 의 값을 diagonal elements 로 지니는 diagonal matrix 를 만듭니다. 

In [4]:
# diagonalize px & py for matrix multiplication
# (4, 4) by (4, 3) by (3, 3) = (4, 3)
np.diag(px)

array([[0.3, 0. , 0. , 0. ],
       [0. , 0.2, 0. , 0. ],
       [0. , 0. , 0.2, 0. ],
       [0. , 0. , 0. , 0.3]])

p(x) 를 곱하는 것이 아니라 나누는 것이기 때문에 역수를 취합니다. 이 때 p(x) 가 0 인 원소는 그 값을 나누지 않고 0 으로 할당합니다.

In [5]:
# inverse elements if the element is greater than 0
np.diag(np.array([0 if pxi == 0 else 1/pxi for pxi in px]))

array([[3.33333333, 0.        , 0.        , 0.        ],
       [0.        , 5.        , 0.        , 0.        ],
       [0.        , 0.        , 5.        , 0.        ],
       [0.        , 0.        , 0.        , 3.33333333]])

위 방법을 이용하여 p(x) 의 역수와 p(y)의 역수로 이뤄진 diagonal matrix 를 만듭니다. 이 때 p(y) 에 a 를 더하는 smoothing 도 할 수 있습니다. p(y)i가 0이 아닐 때 a를 더합니다.

In [6]:
# inverse element digonal matrix of px and py
alpha = 0

px_diag = np.diag(np.array([0 if pxi == 0 else 1/pxi for pxi in px]))
py_diag = np.diag(np.array([0 if pyi == 0 else 1/(pyi + alpha) for pyi in py])) 

print(px_diag.shape)
print(py_diag.shape)

(4, 4)
(3, 3)


행렬 곱은 각 행렬의 .dot 함수를 이용할 수 있습니다. numpy.dot 이 호출되어 두 행렬이 곱해집니다.

In [7]:
# logarithm is not applied yet
exp_pmi = px_diag.dot(pxy).dot(py_diag)
print(exp_pmi)

[[1.66666667 0.         0.        ]
 [0.         5.         0.        ]
 [0.83333333 0.         2.5       ]
 [1.11111111 0.         1.66666667]]


우리가 행렬곱으로 계산한 결과와 손으로 직접 계산한 결과가 같은지 확인합니다. 이처럼 계산과정이 제대로 구현되었는지 값을 넣어 직접 확인하는 작업은 매우 중요합니다.

행렬곱으로 얻은 값과 손으로 계산한 값이 다르면 그 값을 출력하도록 합니다. 네 개의 값이 서로 다릅니다. 하지만 그 값 차이를 살펴보면 float truncated error 때문에 발생한 것임을 알 수 있습니다.

In [8]:
# check value
# difference cause by truncation error
for i in range(x.shape[0]):
    for j in range(x.shape[1]):
        exp_pmi_ij = exp_pmi[i,j]
        manually_ij = pxy[i,j] / (px[i] * py[j])
        if not (exp_pmi_ij == manually_ij):
            print('({}, {}), exp_pmi = {}, manually = {}'.format(i, j, exp_pmi_ij, manually_ij))

(1, 1), exp_pmi = 5.0, manually = 4.999999999999999
(2, 2), exp_pmi = 2.5, manually = 2.4999999999999996
(3, 0), exp_pmi = 1.1111111111111114, manually = 1.1111111111111112
(3, 2), exp_pmi = 1.666666666666667, manually = 1.6666666666666667


log 값을 취해야 합니다. Minimum pmi 보다 작은 경우는 제거하고, (x,y) = pmi 의 형식으로 dok_matrix 에 저장합니다. Sparse matrix 의 형식 중 하나입니다.

numpy.where 를 이용하면 해당 조건을 만족하는 rows, columns 가 return 됩니다. zip(rows, cols) 를 이용하여 각 (i, j)의 값에 접근합니다.

In [11]:
from scipy.sparse import dok_matrix

# PPMI using threshold
min_exp_pmi = 1

rows, cols = np.where(exp_pmi > min_exp_pmi)
pmi_dok = dok_matrix(exp_pmi.shape)

for i, j in zip(rows, cols):
    # apply logarithm
    pmi_dok[i, j] = np.log(exp_pmi[i,j])

In [12]:
for position, pmi_ij in pmi_dok.items():
    print('{} = {} (exp = {})'.format(position, pmi_ij, np.exp(pmi_ij)))

(0, 0) = 0.5108256237659907 (exp = 1.6666666666666667)
(1, 1) = 1.6094379124341003 (exp = 4.999999999999999)
(2, 2) = 0.9162907318741551 (exp = 2.5)
(3, 0) = 0.10536051565782655 (exp = 1.1111111111111114)
(3, 2) = 0.5108256237659908 (exp = 1.666666666666667)


## Implementing PMI with sparse matrix (scipy.sparse)

앞서서 logic을 확인하였으니 이를 응용하여 sparse matrix 에서의 PMI 를 계산하는 과정을 구현합니다.
데이터는 (질문, 답변) pairs 의 단어 간의 co-occurrence matrix 입니다. 질문이 x, 답변이 y 입니다. 17k 개의 단어로 이뤄져 있습니다

In [13]:
idx2vocab = [word.strip() for word in f]
vocab2idx = {vocab:idx for idx, vocab in enumerate(idx2vocab)}
print(x.shape)

NameError: name 'f' is not defined

sparse matrix 에서도 sum(axis=0) 과 sum(axis=1) 은 같습니다. reshape(-1)을 이용하여 row vector 를 만듭니다.

In [ ]:
# convert x to probability matrix & marginal probability
px = (x.sum(axis=1) / x.sum()).reshape(-1)
py = (x.sum(axis=0) / x.sum()).reshape(-1)
pxy = x/x.sum()

print(px.shape)
print(py.shape)
print(pxy.shape)

rows 를 list로 만든 뒤, 이를 diagonal elements로 지니는 diagonal matrix 로 만듭니다. scipy.sparse 에서도 diagonal matrix를 제공합니다.

In [ ]:
from scipy.sparse import diags

px_diag = diags(px.tolist()[0])
py_diag = diags(py.tolist()[0])

In [ ]:
scipy.sparse.diag 의 data는 numpy.ndarray 입니다. Diagonal matrix는 대각의 원소만 저장하면 됨

## 1

In [ ]:
import numpy as np

In [21]:
def preprocess(text):
    text = text.lower() # 전부 소문자로
    text = text.replace('.', ' .') # 마침표 replace
    words = text.split(' ') # ' ' 로 split
    word_to_id = {}
    id_to_word = {}
    for word in words:
        if word not in word_to_id: # word_to_id 라는 dict에 존재하지 않으면
            new_id = len(word_to_id) # idx 부여
            word_to_id[word] = new_id # key 값과 value 값 정의
            id_to_word[new_id] = word # key 값과 value 값 정의
    corpus = np.array([word_to_id[w] for w in words]) # word_to_id 의 value 값들 행렬로 만듬
    return corpus, word_to_id, id_to_word



In [22]:
text = "You say goodbye and I say hello."
corpus, word_to_id, id_to_word = preprocess(text)

##  2

In [23]:
print(corpus)
print(word_to_id)
print(id_to_word)

[0 1 2 3 4 1 5 6]
{'you': 0, 'say': 1, 'goodbye': 2, 'and': 3, 'i': 4, 'hello': 5, '.': 6}
{0: 'you', 1: 'say', 2: 'goodbye', 3: 'and', 4: 'i', 5: 'hello', 6: '.'}


## 3

In [24]:
def create_co_matrix(corpus, vocab_size, window_size=1):
    corpus_size = len(corpus)
    co_matrix = np.zeros((vocab_size, vocab_size), dtype=np.int32)
    for idx, word_id in enumerate(corpus):
        for i in range(1, window_size + 1):
            left_idx = idx - i
            right_idx = idx + i
            if left_idx >= 0: # idx가 0이 아닐 때를 제외
                left_word_id = corpus[left_idx]
                co_matrix[word_id, left_word_id] += 1
            if right_idx < corpus_size: # idx가 맨 끝이 아닐 때를 제외
                right_word_id = corpus[right_idx]
                co_matrix[word_id, right_word_id] += 1
    return co_matrix

In [25]:
create_co_matrix(corpus, len(corpus), 1)

array([[0, 1, 0, 0, 0, 0, 0, 0],
       [1, 0, 1, 0, 1, 1, 0, 0],
       [0, 1, 0, 1, 0, 0, 0, 0],
       [0, 0, 1, 0, 1, 0, 0, 0],
       [0, 1, 0, 1, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0]])

## 4

코사인 유사도

In [26]:
def cos_similarity(x,y, eps = 1e-8):
    nx = x/np.sqrt(np.sum(x**2) + eps) #x의 정규화
    ny = y/np.sqrt(np.sum(y**2) + eps) #y의 정규화
    return np.dot(nx,ny)

In [29]:
#import sys
#sys.path.append('..')
text = 'You say goodbye and I say hello'
corpus, word_to_id, id_to_word = preprocess(text)
vocab_size = len(word_to_id)
C = create_co_matrix(corpus, vocab_size)
c0 = C[word_to_id['you']] #'you'의 단어 벡터
c1 = C[word_to_id['i']] #'i'의 단어 벡터
print(c0, c1)
print(cos_similarity(c0, c1))

[0 1 0 0 0 0] [0 1 0 1 0 0]
0.7071067758832467


## 5

In [31]:
def most_similar(query, word_to_id, id_to_word, word_matrix, top=5):
#query(검색 단어) 검색
    if query not in word_to_id:
        print('%s(을)를 찾을 수 없습니다.' % query)
        return
#word_matrix(단어 벡터)안에서 query id 찾기
    print('\n[query] ' + query)
    query_id = word_to_id[query]
    query_vec = word_matrix[query_id]

#단어 벡터 안 단어들을 코사인 유사도 계산
    vocab_size = len(id_to_word)
    similarity = np.zeros(vocab_size)
    for i in range(vocab_size):
        similarity[i] = cos_similarity(word_matrix[i], query_vec)

#코사인 유사도를 기준으로 내림차순 출력
    count = 0
    for i in (-1 * similarity).argsort():
        if id_to_word[i] == query:
            continue
        print(' %s : %s' % (id_to_word[i], similarity[i]))
        count += 1
        if count >= top:
            return

## 6

단어 앞뒤에 무엇이 있는가에 대해 벡터간 유사도를 판단한 것이라, 정확히 단어의 본질을 파악한 것이 아니라 생긴 문제

In [32]:
text = 'You say goodbye and I say hello.'
corpus, word_to_id, id_to_word = preprocess(text)
vocab_size = len(word_to_id)
C = create_co_matrix(corpus, vocab_size)

most_similar('you', word_to_id, id_to_word, C, top=5)


[query] you
 goodbye : 0.7071067758832467
 i : 0.7071067758832467
 hello : 0.7071067758832467
 say : 0.0
 and : 0.0


[통계 기반 기법 개선]

앞의 통계 기반 기법은, 인접한 단어가 얼마나 많은가에 대한 유사도를 사용하는 것인데,
발생 횟수라는 것은 그리 좋은 '특징'이 아니다.
the car 이라는 단어는,
the 와 car 의 단어가 언어 문법상 동시 발생 빈도가 높은데,
the 와 car 의 유사도가, drive와 같은, 정말 car 와 입접한 단어보다 높다.

하지만 the 는 car 가 아님

이를 해결하기 위해 사용하는 척도가 있다.
'점별 상호정보량 (Pointwise Mutual Information : PMI)

(상호 정보량)
PMI 수식 : 확률변수 x와 y에 대한 PMI
PMI(x, y) = log2 P(x,y)/P(x)P(y)
: P(x)는 x가 일어날 확률, P(y)는 y가 일어날 확률, P(x,y) 는 x와 y가 동시에 발생할 확률을 말합니다.
PMI가 높을수록 두 확률변수의 관련성이 높다는 의미 

앞의 자연어 예에 적용하면, P(x) 는 단어 x가 말뭉치에 등장할 확률, P(y) 는 단어 y가 등장할 확률
위의 수식을 사용한다면 말뭉치가 적더라도, 제법 제대로된 유사도를 추출할 수 있을 것.

또한 단순히 단어 주변의 단어의 발생 빈도만을 파악하는 유사도 방식보다는, 총 발생 빈도에 대해 같이 사용된 빈도에 대한 비율로
유사도를 파악하기에, the 와 같이 단순히 발생 빈도가 높은 것에 유사도를 높이는 것이 아니고,
빈도는 낮아도, 같이 사용될 일이 많은 단어끼리 유사도를 높일 수 있음

PMI 개선 : 만약 두 단어 사이에 동시 발생 횟수가 0 이면 log2 0 이 되버리므로
'양의 상호정보량 (Positive PMI: PPMI)' 를 사용함

PPMI(x,y) = max(0, PMI(x,y))

max 함수를 사용하면, 둘 사이에 큰 값을 가져오게 되기에, 만약 PMI 가 -infinity 가 되면 0 이 되는 것

## 7

verbose 는 진행상황 출력 여부, eps 

In [33]:
def ppmi(C, verbose = False, eps = 1e-8):
    M = np.zeros_like(C, dtype=np.float32)
    N = np.sum(C)
    S = np.sum(C, axis=0)
    total = C.shape[0] * C.shape[1]
    cnt = 0
    for i in range(C.shape[0]):
        for j in range(C.shape[1]):
            pmi = np.log2(C[i,j]*N/(S[j]*S[i])+eps)
            M[i,j] = max(0,pmi)
            if verbose:
                cnt += 1
                if cnt %(total//100) == 0:
                    print('%.1f%% 완료' % (100*cnt/total))
    return M

## 8 

In [34]:
import numpy as np

text = 'You say goodbye and I say hello'
corpus, word_to_id, id_to_word = preprocess(text)
vocab_size = len(word_to_id)
C = create_co_matrix(corpus, vocab_size)
W = ppmi(C)
np.set_printoptions(precision=3) # 유효 자릿수를 세자리로 표시
print('동시발생 행렬')
print(C)
print('-'*50)
print('PPMI')
print(W)

동시발생 행렬
[[0 1 0 0 0 0]
 [1 0 1 0 1 1]
 [0 1 0 1 0 0]
 [0 0 1 0 1 0]
 [0 1 0 1 0 0]
 [0 1 0 0 0 0]]
--------------------------------------------------
PPMI
[[0.    1.585 0.    0.    0.    0.   ]
 [1.585 0.    0.585 0.    0.585 1.585]
 [0.    0.585 0.    1.585 0.    0.   ]
 [0.    0.    1.585 0.    1.585 0.   ]
 [0.    0.585 0.    1.585 0.    0.   ]
 [0.    1.585 0.    0.    0.    0.   ]]


하지만 위와 같은 PPMI도 문제가 있음
바로 메모리 문제
말뭉치의 어휘 수에 따라 벡터의 차원수가 증가한다
말뭉치가 커질수록 현실적으로 벡터를 저장한 메모리가 필요해진다

또한 낭비되는 정보가 많은데, 근처 두 단어만 서로 비교하고, 나머지는 모두 0 인데, 의미있는 유사점 외에는 대다수가 0으로, 의미가 없는 정보

자연어 처리만이 아니라, 데이터에서 정보를 파악하는 데이터 사이언스 분야에서, 이러한 의미없는 정보를 줄이고 데이터에서 중요한 정보를 추출해내 사용하는 방식은 널리 쓰임.

'벡터의 차원감소'